## Face Recognition – Launch a webserver when a face is recongnized and send alert 


### Step 1 - Create Training Data

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(1)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Step 2 - Training the Model with precreated dataset

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

#Creating a function that trains a model to recognise certain faces provided the dataset 
def trainmodel(name , path) : 
    data_path = path
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
    Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)

# Create a numpy array for both training data and labels
    Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

    model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
    model.train(np.asarray(Training_Data), np.asarray(Labels))
    print(str(name) + "'s" + " Model trained sucessefully")
    return model


In [2]:
Yashwanth_Model = trainmodel("Yashwanth" , './faces/yash/')  #Passing the dataset-1 to the trainmodel function 

Yashwanth's Model trained sucessefully


In [3]:
Amma_Model = trainmodel("Amma" , './faces/Amma/')   #Passing the dataset-2 to the trainmodel function 

Amma's Model trained sucessefully


In [4]:
#Function that send's mail to a person along with the image when a particular face is detected

def mail(sender , receiver , passwd , img) :
    import smtplib
    import imghdr
    from email.message import EmailMessage

    Sender_Email = sender
    Reciever_Email = receiver
    Password = passwd

    newMessage = EmailMessage()                         
    newMessage['Subject'] = "There's an Intruder" 
    newMessage['From'] = Sender_Email                   
    newMessage['To'] = Reciever_Email                   
    newMessage.set_content('Imge of the Intruder is Attached!!') 

    with open(img, 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    
        smtp.login(Sender_Email, Password)              
        smtp.send_message(newMessage)

In [5]:
#Functions capable of sending a WhatsApp message when called 
def get_time():
    import time
    t = time.localtime()
    hour = int(time.strftime("%H", t))
    min = int(time.strftime("%M", t)) +1 
    return hour, min
def send_whatsApp_msg(to , text):
    import pywhatkit
    pywhatkit.sendwhatmsg(to,text, get_time()[0], get_time()[1]) 

### Step 3 - Run Our Facial Recognition

In [ ]:
import cv2
import numpy as np
import os


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#Determining the confidence of prediction by the ML model 
def confidence(results , image):
    if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
    return confidence
        

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(1)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        yash_results = Yashwanth_Model.predict(face)
        amma_results = Amma_Model.predict(face)
        # harry_model.predict(face)
        
        yash_conf = confidence(yash_results , image)
        amma_conf = confidence(amma_results , image)
        
        if yash_conf > 85:
            cv2.putText(image, "Hey Yash", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            display_string = str(yash_conf) + '% Confident it is User'
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition', image)
            try :
                cv2.imwrite("Intruder.jpg" , image)
                import getpass
                print("Ready to send the mail , Please provide the required credentials....")
                sender = input("Enter your mailID : ")
                receiver = input("Enter the receiver's mailID : ")
                password = getpass.getpass("Enter your password : ")
                mail(sender , receiver , password , "Intruder.jpg")
                print("Mail sent !!")
                send_whatsApp_msg("+917036970919" , "You have been detected")
                print("Message sent on Whatsapp !!")
                break
                    
            except :
                print("Please check the availability of third party applications to your MailID")
            
        elif amma_conf > 80 :
            cv2.putText(image, "Hey Mom", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            display_string = str(amma_conf) + '% Confident it is User'
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition', image )
            try :
                import os
                print("Provisioning the environment ...")
                os.system("terraform init")
                os.system("terraform apply -auto-approve")
                print("OS provisioned , wait for the OS to be configured.....")
                os.system("pscp -pw Yashwanth@3 -batch ip root@192.168.0.13:/root/mlops/")
                os.system("plink root@192.168.0.13 -batch -pw Yashwanth@3 ansible-playbook /root/mlops/script.yml")
                print("OS configured !!! You can now access the webpage ")
            except:
                print("Check the code properly")
            
         
        else:
            
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

Provisioning the environment ...
OS provisioned , wait for the OS to be configured.....
OS configured !!! You can now access the webpage 
Ready to send the mail , Please provide the required credentials....
